# Crosstalk Analysis and Strip Plotter

### Note: Use the epi-paint kernel only

In [ ]:
# Import dependencies

import pandas as _pd
import os.path as _ospath
import numpy as _np
import os as _os
import seaborn as _sns
import matplotlib.pyplot as _plt
import joypy
import colorsys

In [ ]:
# Define the folder where the data file is there.

folder = '/Users/abhinav/Library/CloudStorage/OneDrive-IndianInstituteofScience/Papers/zz_Msequences/Multiplexing/20250116'     # Directory with data file
data_file = 'data_individual.csv'                                                                                               # Define file name
data_path = _ospath.join(folder, data_file)                                                                                     # Build the path

In [ ]:
data = _pd.read_csv(data_path)
order = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12', 'Random']  # Define the order of the channels for plotting
data['Channel'] = _pd.Categorical(data['Channel'], categories=order, ordered=True)  # Set the order of the 'Channel' column
data = data.sort_values('Channel')
data['Pick'] = data['Pick'].replace('random', 'Random')

In [ ]:
for channel in order:
    data_channel = data[data['Channel'] == channel]
    joypy.joyplot(
        data_channel,
        by='Pick',
        column='Count',
        colormap=_plt.cm.Set2,
        fade=True,
        figsize=(8,6),
        overlap=1
    )
    _plt.xlim(0, 10)
    _plt.xlabel('Count')
    _plt.title(f"Ridge Plot of Counts per Pick for {channel}")
    _plt.show()

In [ ]:
data['Pick'].unique()

In [ ]:
hues = _np.arange(0, 1, 1 / 12)
colors = [colorsys.hsv_to_rgb(_, 1, 1) for _ in hues]
colors = colors[-6:] + colors[:-6]
color_map = dict(zip(order, colors))
color_map['Random'] = 'k'

for channel in order:
    data_channel = data[data['Channel'] == channel]
    ref_mean = data_channel[data_channel['Pick'] == channel]['Count'].mean()
    threshold = ref_mean * 0.11  # 11% of the mean count for the correct channel

    # Create a temporary palette: current channel in its color, others gray
    temp_palette = {}
    for ch in order:
        if ch == channel:
            temp_palette[ch] = color_map[ch]
        elif ch == 'Random':
            temp_palette[ch] = 'k'
        elif data_channel[data_channel['Pick'] == ch]['Count'].mean() > threshold:
            temp_palette[ch] = color_map[ch]
        else:
            temp_palette[ch] = 'gray'

    _plt.figure(figsize=(10, 1.5))

    # _sns.violinplot(data = data_channel, x = 'Pick', y = 'Count', hue = 'Pick', order = order, palette = color_map, linewidth=0, bw_adjust=1.5, inner = 'box')
    _sns.stripplot(data = data_channel, x = 'Pick', y = 'Count', hue = 'Pick', order = order, palette=temp_palette, size=3, alpha=0.03, jitter=True, rasterized=True)
    _sns.boxplot(data = data_channel, x = 'Pick', y = 'Count', order = order, fill = True, whis = 1.5, width=0.2, showcaps=False, boxprops={'facecolor':'none', "zorder":10}, showfliers=False, whiskerprops={'linewidth':0.5, "zorder":10}, medianprops={'color':'black', 'linewidth':2, "zorder":10})
    # _plt.ylabel('Localizations on Origami')
    # _plt.xlabel('Origami Identity')
    # _plt.title(f'Imager {channel}')
    correct_channel_count = data_channel[data_channel['Pick'] == channel]
    if correct_channel_count['Count'].mean() < 500:
        _plt.ylim([0, 1000])
    else:
        _plt.ylim([0, 2000])
    mean_val = correct_channel_count["Count"].mean() * 0.11
    _plt.axhline(y=mean_val, color='k', dashes = (3, 2), linewidth=1)
    # _plt.xticks(rotation=45)
    _plt.xlabel('')
    _plt.ylabel('')
    print(f'Mean Count for {channel}: {correct_channel_count["Count"].mean()}')
    save_path = _ospath.join(folder, f'violin_plot_{channel}.svg')
    _plt.savefig(save_path, format='svg', bbox_inches='tight')
    _plt.show()

In [ ]:
for channel in order:
    data_channel = data[data['Channel'] == channel]
    g = _sns.FacetGrid(data_channel, row = 'Pick', row_order = order, aspect = 15, sharex=True, height = 1.5)
    g.map(_sns.kdeplot, 'Count', fill = True, color = 'blue', bw_adjust = 1.5)
    g.set_titles(row_template = '{row_name}')
    g.set(ylabel = '', yticks = [])
    g.despine(bottom=False, left=True)
    _plt.xlim([-100,2000])
    _plt.show()